<a href="https://colab.research.google.com/github/Anisha234/Identifying-Tissue-of-Origin-From-miRNA/blob/main/SRA_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Notebook to pre-process miRNA data from blood from the eVmiRNA database
import pandas as pd
import numpy as np
import os
import sys

In [39]:
df = pd.read_csv('/content/miTED-results_long.tsv',delimiter='\t')
df_ref_data = pd.read_csv('/content/full_miRNA_metadata_dataset_unnorm.csv')

<ipython-input-39-0e377be59f6f>:2: DtypeWarning: Columns (564) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ref_data = pd.read_csv('/content/full_miRNA_metadata_dataset_unnorm.csv')


In [40]:
print(df.shape)
df=df.drop(columns=['S.No', 'Sample_ID', 'Collection', 'Project_ID', 'Tissue_subregion', 'Cell_Line', 'Organism', 'Health_state', 'Tissue_definition','Disease','Gender'])
df.replace('male', 1, inplace=True)
df.replace('female', 0, inplace=True)
df.rename(columns={'Tissue_or_organ_of_origin':'disease_type', 'Gender':'gender'}, inplace=True)

disease_labels=["Breast", "Uterus", "Ovary", "Prostate", "Testis" ,"Lung", "Kidney","Bladder","Esophagus", "Liver","Pancreas","Pleura","Colon", "Skin", "Stomach",
                 "Brain", "Cervix", "Thyroid"]
df=df.loc[df['disease_type'].isin(disease_labels)]

df.replace(disease_labels,list(range(len(disease_labels))),
              inplace=True)


df=df.reset_index(drop=True)
df.rename(columns={'Tissue_or_organ_of_origin':'disease_type', 'Gender':'gender'}, inplace=True)
print(df.columns)
df_feat=df

(207, 2668)
Index(['disease_type', 'hsa-let-7a-2-3p', 'hsa-let-7a-3p', 'hsa-let-7a-5p',
       'hsa-let-7b-3p', 'hsa-let-7b-5p', 'hsa-let-7c-3p', 'hsa-let-7c-5p',
       'hsa-let-7d-3p', 'hsa-let-7d-5p',
       ...
       'hsa-miR-9901', 'hsa-miR-9902', 'hsa-miR-9903', 'hsa-miR-9983-3p',
       'hsa-miR-9985', 'hsa-miR-9986', 'hsa-miR-99a-3p', 'hsa-miR-99a-5p',
       'hsa-miR-99b-3p', 'hsa-miR-99b-5p'],
      dtype='object', length=2657)


In [41]:
feat_ref = set(df_ref_data.columns)
feat_SRA =list(df_feat.columns)
print(len(feat_ref), len(feat_SRA))

# Replace miR with mir
feat_clean = feat_SRA
feat_to_remove=[]
mapping_dict={}
for i,element in enumerate(feat_SRA):
    orig_element = element
    element= element.replace("miR", "mir")
    element= element.replace('-3p', "")
    element= element.replace('-5p', "")
    feat_clean[i] = element
    if element not in feat_ref:
        feat_to_remove.append(orig_element)
    else:
        mapping_dict[orig_element] = element

print('Before',len(feat_clean))
feat_clean = set(feat_clean)
print('After', len(feat_clean))
#print(feat_clean, len(feat_clean))
#print(mapping_dict)

567 2657
Before 2657
After 1793


In [42]:
common_feat= feat_clean.intersection(feat_ref)
print(len(common_feat))


498


In [43]:
#Now rename columns
df_filtered =df_feat[list(mapping_dict.keys())]

rename_dict={}
duplicate_dict={}
for col in df_filtered.columns:
    new_col = col.replace('miR', 'mir')
    new_col= new_col.replace('-3p', "")
    new_col= new_col.replace('-5p', "")
    if new_col in rename_dict.values():
        duplicate_dict[col] = new_col
    else:
        rename_dict[col] = new_col
print('Rename dict', len(rename_dict))
print(rename_dict)
print('Duplicate dict', len(duplicate_dict))
print(duplicate_dict)
df_filtered.rename(rename_dict, axis=1, inplace=True)
#print(df_filtered.head())
#print(len(set(df_filtered.columns)))
# Now look at the columns in duplicate_dict and add the values there to the new_col
# Then remove the columns in duplicate_dict (i.e the keys)

for col, new_col in duplicate_dict.items():
    df_filtered[new_col] = df_filtered[new_col]+ df_filtered[col]

df_filtered.drop(columns=duplicate_dict.keys(), inplace=True)
#df_filtered['case_id'] = df_filtered.index
df_filtered= df_filtered.reset_index() #drop=True)
df_filtered.rename(columns={'index':'case_id'}, inplace=True)
print(df_filtered.columns)

Rename dict 498
{'disease_type': 'disease_type', 'hsa-let-7a-2-3p': 'hsa-let-7a-2', 'hsa-let-7b-3p': 'hsa-let-7b', 'hsa-let-7c-3p': 'hsa-let-7c', 'hsa-let-7d-3p': 'hsa-let-7d', 'hsa-let-7e-3p': 'hsa-let-7e', 'hsa-let-7f-1-3p': 'hsa-let-7f-1', 'hsa-let-7f-2-3p': 'hsa-let-7f-2', 'hsa-let-7g-3p': 'hsa-let-7g', 'hsa-let-7i-3p': 'hsa-let-7i', 'hsa-miR-100-3p': 'hsa-mir-100', 'hsa-miR-101-2-5p': 'hsa-mir-101-2', 'hsa-miR-103a-1-5p': 'hsa-mir-103a-1', 'hsa-miR-103a-2-5p': 'hsa-mir-103a-2', 'hsa-miR-106a-3p': 'hsa-mir-106a', 'hsa-miR-106b-3p': 'hsa-mir-106b', 'hsa-miR-107': 'hsa-mir-107', 'hsa-miR-10a-3p': 'hsa-mir-10a', 'hsa-miR-10b-3p': 'hsa-mir-10b', 'hsa-miR-1180-3p': 'hsa-mir-1180', 'hsa-miR-1181': 'hsa-mir-1181', 'hsa-miR-1224-3p': 'hsa-mir-1224', 'hsa-miR-1226-3p': 'hsa-mir-1226', 'hsa-miR-1228-3p': 'hsa-mir-1228', 'hsa-miR-1229-3p': 'hsa-mir-1229', 'hsa-miR-1245a': 'hsa-mir-1245a', 'hsa-miR-1247-3p': 'hsa-mir-1247', 'hsa-miR-1248': 'hsa-mir-1248', 'hsa-miR-1249-3p': 'hsa-mir-1249', 'hs

<ipython-input-43-3acba484fcff>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(rename_dict, axis=1, inplace=True)
<ipython-input-43-3acba484fcff>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[new_col] = df_filtered[new_col]+ df_filtered[col]


Index(['case_id', 'disease_type', 'hsa-let-7a-2', 'hsa-let-7b', 'hsa-let-7c',
       'hsa-let-7d', 'hsa-let-7e', 'hsa-let-7f-1', 'hsa-let-7f-2',
       'hsa-let-7g',
       ...
       'hsa-mir-937', 'hsa-mir-939', 'hsa-mir-940', 'hsa-mir-942',
       'hsa-mir-944', 'hsa-mir-95', 'hsa-mir-96', 'hsa-mir-98', 'hsa-mir-99a',
       'hsa-mir-99b'],
      dtype='object', length=499)


<ipython-input-43-3acba484fcff>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=duplicate_dict.keys(), inplace=True)


In [44]:

df_final=df_filtered.dropna()
column_names = pd.DataFrame(df_final.columns, columns=["column"])
column_names.to_csv('miRNA_SRA_features.csv', index=False)
df_final.drop(columns=['case_id'],inplace=True)
df_final.to_csv('SRA_data.csv', index=False)